In [15]:
import asyncio
import websockets
import json
import pandas as pd
from time import sleep
import pprint
import hmac
import hashlib
from datetime import datetime
import nest_asyncio
import numpy as np
import matplotlib.pyplot as plt
import time
from scipy.stats import norm

nest_asyncio.apply()

In [73]:
class MM_robot:
    def __init__(self, instrument, currency, url, clientId, clientSecret, sleep_time):
        self.url = url
        self.clientId = clientId
        self.clientSecret = clientSecret
        self.instrument = instrument
        self.currency = currency
        self.order = None
        self.order_id = None
        self.order_side = self.get_side()
        self.order_price = self.get_price()
        self.d1 = 0
        self.d2 = 0
        self.S0 = self.get_S()
        self.S = self.S0
        self.K_call, self.K_put = self.get_strikes()
        self.delta_call = 0
        self.delta_put = 0
        self.r = 0
        self.tau = 1/12
        self.q = 0
        self.sleep_time = sleep_time
        self.sigma = 0
        self.position_signal = 0
        self.current_position = self.get_current_position()
        self.data_dict = {"time": [], "sigma": [], "delta_call": [], "delta_put": [], "current_position": [], "new_signal_is": [], "mark_price": []}
        
    async def pub_api(self,msg):
        """Функция для выгрузки публичных данных."""
        async with websockets.connect(self.url) as websocket:
            await websocket.send(msg)
            while websocket.open:
                response = await websocket.recv()
                return json.loads(response)
        
    async def priv_api(self,msg):
        """Функция для выгрузки приватных данных."""
        async with websockets.connect(self.url) as websocket:
            auth_msg = {
            "jsonrpc":"2.0",
            "id":42,
            "method":"public/auth",
            "params":{
            "grant_type":"client_credentials",
            "client_id" : self.clientId,
            "client_secret" : self.clientSecret
                }
    
            }
            await websocket.send(json.dumps(auth_msg))
            while websocket.open:
                response = await websocket.recv()
                await websocket.send(msg)
                response = await websocket.recv()
                break
            return json.loads(response)
                
    def order_msg(self, price, side, weight):
        return {
          "jsonrpc" : "2.0",
          "id" : 42,
          "method" : "private/"+side,
          "params" : {
            "instrument_name" : self.instrument,
            "amount" : round(weight),
            "type" : "limit",
            "label" : "order_"+side,
            "price" : price,
            "time_in_force": "good_til_cancelled",
            "post_only": True
          }
        }
        
    def normal_distribution_value(self,x): 
        """Функция для подсчета кумулятивного нормального распределения в точке x."""
        return norm.cdf(x, 0, 1) 
    
    def get_d1(self,S, K, r, q, sigma, tau): 
        """Функция для расчёта d1."""
        return (np.log(S / K) + (r - q + 0.5 * sigma ** 2) * tau) / (sigma * np.sqrt(tau))
        
    def get_d2(self,S, K, r, q, sigma, tau):
        """Функция для расчёта d2."""
        return (np.log(S / K) + (r - q - 0.5 * sigma ** 2) * tau) / (sigma * np.sqrt(tau))
 
    def get_S(self): 
        """Функция для получения цены фьючерса."""
        order_book_msg = {
        "jsonrpc" : "2.0",
        "id" : 42,
        "method" : "public/get_order_book",
        "params" : {
            "instrument_name" : self.instrument,
            "depth" : 10
            }
        }
        s = asyncio.get_event_loop().run_until_complete(self.pub_api(json.dumps(order_book_msg)))['result']['mark_price']
        return s
        
    def get_volatility(self):
        """Функция для расчёта волатильности."""
        vol_msg =  {
        "jsonrpc" : "2.0",
        "id" : 42,
        "method" : "public/get_historical_volatility",
        "params" : {
        "currency" : self.currency
            }
        }
        result = asyncio.get_event_loop().run_until_complete(self.pub_api(json.dumps(vol_msg)))['result']
        df = pd.DataFrame(result)
        vt = df.ewm(span=14, adjust=False).mean()
        return vt[1].iloc[-1] /100
        
    def get_delta_call(self, S, K, r, q, sigma, tau):
        """Функция для расчёта колл-дельты."""
        return self.normal_distribution_value(self.get_d1(S, K, r, q, sigma, tau))
    
    def get_delta_put(self, S, K, r, q, sigma, tau):
        """Функция для расчёта пут-дельты."""
        return -self.normal_distribution_value(-1 * self.get_d1(S, K, r, q, sigma, tau))
        
    def get_expiration_date(self):
        """Функция для получения времени до экспирации фьючерса."""
        expiration_msg = \
        {
        "method" : "public/get_instrument",
        "params" : {
            "instrument_name" : self.instrument
        },
        "jsonrpc" : "2.0",
        "id" : 2
        }
        exp_date_timestamp = asyncio.get_event_loop().run_until_complete(self.pub_api(json.dumps(expiration_msg)))['result']['expiration_timestamp']
        exp_date_timestamp = exp_date_timestamp/1000
        return (exp_date_timestamp - (datetime.now().timestamp())) / (3600*24*365)
        
    def get_tau(self):
        self.tau = self.tau - self.sleep_time / (365*24*60*60)
        return self.tau
    
    def get_strikes(self):
        """Функция для высчитывания страйков двух потенциально имеющихся фьючерсов."""
        K_call = 1.01 * self.S0
        K_put = self.S0
        return K_call, K_put
    
    def update(self):
        """Функция для обновления данных."""
        self.tau = self.get_tau() # 0 запрос
        self.sigma = self.get_volatility() # 1 запрос
        self.S = self.get_S() # 1 запрос 
        self.delta_call = self.get_delta_call(self.S, self.K_call, self.r, self.q, self.sigma, self.tau) # 0 запрос
        self.delta_put = self.get_delta_put(self.S, self.K_put, self.r, self.q, self.sigma, self.tau)  # 0 запрос
        self.current_position = self.get_current_position() # 1 запрос
        self.position_signal = self.get_position_signal() # 0 запрос
        self.data_dict['time'].append(time.ctime())
        self.data_dict['sigma'].append(self.sigma)
        self.data_dict['delta_call'].append(self.delta_call)
        self.data_dict['delta_put'].append(self.delta_put)
        self.data_dict['current_position'].append(self.current_position)
        self.data_dict['new_signal_is'].append(-self.position_signal)
        self.data_dict['mark_price'].append(self.S)
    
    def get_position_signal(self):
        """Функция получения сигнала."""
        return self.delta_call + self.delta_put  + self.current_position
    
    def get_bid_ask(self): 
        """Функция для тороговли."""
        msg = { 
            "jsonrpc" : "2.0", 
            "id" : 3659, 
            "method" : "public/get_book_summary_by_instrument", 
            "params" : { 
            "instrument_name" : self.instrument 
            } 
        } 
        result = asyncio.get_event_loop().run_until_complete(self.pub_api(json.dumps(msg)))['result'][0] 
        return result['bid_price'], result['ask_price']

    def cancel_order(self, order_id):
        """Функция для снятия ордера из стакана."""
        msg = \
        {
          "jsonrpc" : "2.0",
          "id" : 42,
          "method" : "private/cancel",
          "params" : {
            "order_id" : order_id
          }
        }
        canceld_position = asyncio.get_event_loop().run_until_complete(self.priv_api(json.dumps(msg)))
        
    def has_active_order(self):
        """Возвращает True, если в стакане есть открытая заявка и False, если открытых заявок нет"""
        get_open_orders_msg = \
        {
        "jsonrpc" : "2.0",
        "id" : 42,
        "method" : "private/get_open_orders_by_instrument",
        "params" : {
            "instrument_name" : self.instrument
        }
        }
        res = asyncio.get_event_loop().run_until_complete(self.priv_api(json.dumps(get_open_orders_msg)))['result']
        if res == []:
            return False
        else:
            return True
            
    def get_id(self):
        """Возвращает id ордера."""
        get_open_orders_msg = \
        {
        "jsonrpc" : "2.0",
        "id" : 42,
        "method" : "private/get_open_orders_by_instrument",
        "params" : {
            "instrument_name" : self.instrument
            }
        }
        try: 
            return asyncio.get_event_loop().run_until_complete(self.priv_api(json.dumps(get_open_orders_msg)))['result'][0]['order_id']
        except:
            return None

    def get_price(self):
        """Возвращает id ордера."""
        get_open_orders_msg = \
        {
        "jsonrpc" : "2.0",
        "id" : 42,
        "method" : "private/get_open_orders_by_instrument",
        "params" : {
            "instrument_name" : self.instrument
            }
        }
        try: 
            return asyncio.get_event_loop().run_until_complete(self.priv_api(json.dumps(get_open_orders_msg)))['result'][0]['price']
        except:
            return None
            
    def get_amount(self):
        """Возвращает id ордера."""
        get_open_orders_msg = \
        {
        "jsonrpc" : "2.0",
        "id" : 42,
        "method" : "private/get_open_orders_by_instrument",
        "params" : {
            "instrument_name" : self.instrument
            }
        }
        try: 
            return asyncio.get_event_loop().run_until_complete(self.priv_api(json.dumps(get_open_orders_msg)))['result'][0]['amount']
        except:
            return None  
            
    def get_side(self):
        """Возвращает id ордера."""
        get_open_orders_msg = \
        {
        "jsonrpc" : "2.0",
        "id" : 42,
        "method" : "private/get_open_orders_by_instrument",
        "params" : {
            "instrument_name" : self.instrument
            }
        }
        try: 
            return asyncio.get_event_loop().run_until_complete(self.priv_api(json.dumps(get_open_orders_msg)))['result'][0]['direction']
        except:
            return None  
            
    def edit_order(self, order_id, amount, price):
        """Функция для редактирования имеющейся заявки."""
        msg = {
            "jsonrpc" : "2.0",
            "id" : 42,
            "method" : "private/edit",
            "params" : {
              "order_id" : order_id,
              "amount" : amount,
              "price" : price,
            }
          }
        return asyncio.get_event_loop().run_until_complete(self.priv_api(json.dumps(msg)))
    
    def get_current_position(self): 
        """Возвращает текущую позицию по инструменту в доле от биткоина."""
        position_msg = { 
            "jsonrpc" : "2.0", 
            "id" : 42, 
            "method" : "private/get_position", 
            "params" : { 
            "instrument_name" : self.instrument 
            } 
        } 
        try:
            return asyncio.get_event_loop().run_until_complete(self.priv_api(json.dumps(position_msg)))['result']['size_currency']
        except: 
            return 0 

    def contract_format(self,number):
        """Функция для корректировки формата цены.(Костыльная)"""
        if self.currency == 'BTC':
            number = round(number)
            last_digit = number % 10
            if last_digit >= 5:
                number = number + (10 - last_digit)
            else:
                number = number - last_digit
            return max(number, 10)
        elif self.currency == 'ETH':
            number = round(number)
            return max(number, 1)
        else:   
            number = round(number)
            last_digit = number % 10
            if last_digit >= 5:
                number = number + (10 - last_digit)
            else:
                number = number - last_digit
            return max(number, 10)
    
    def analyze_order(self):
        """Функция для обработки открытого ордера после времени ожидания"""
        if self.position_signal > 0:
            signal_side = "sell"
        else:
            signal_side = "buy"
        order_side = self.order_side
        self.order_id = self.get_id()
        best_bid, best_ask = self.get_bid_ask()
        if signal_side != order_side:
            print('Получил инверсивный сигнал, отменяю текущий ордер!\n')
            self.cancel_order(self.order_id)
            if signal_side == "buy":
                print('После отмены получил сигнал на покупку\n')
                self.order = asyncio.get_event_loop().run_until_complete(self.priv_api(json.dumps(self.order_msg(best_bid, signal_side, self.contract_format(self.S * np.abs(self.position_signal))))))
                self.order_side = "buy"
                self.order_price = best_bid
            else:
                print('После отмены получил сигнал на продажу\n')
                self.order = asyncio.get_event_loop().run_until_complete(self.priv_api(json.dumps(self.order_msg(best_ask, signal_side, self.contract_format(self.S * self.position_signal)))))
                self.order_side = "sell"
                self.order_price = best_ask
        elif signal_side == "sell":
            print('Получил сигнал в том же направлении на продажу\n')
            if self.order_price > best_ask:
                print('Получил приказ переставить заявку на best_ask!\n')
                self.order = self.edit_order(self.order_id, self.contract_format(self.S * self.position_signal), best_ask)
                self.order_price = best_ask
        elif signal_side == "buy":
            print('Получил сигнал в том же направлении на покупку\n')
            if self.order_price < best_bid:
                print('Получил приказ переставить заявку на best_bid!\n')
                self.order = self.edit_order(self.order_id, self.contract_format(self.S * np.abs(self.position_signal)), best_bid)
                self.order_price = best_bid
            
    def send_order(self):
        """Функция отправки ордера."""
        if self.has_active_order():
            self.analyze_order()
            print('Analyzing order...\n')
        elif self.position_signal > 0:
            print('Have no orders, get signal to sell!\n')
            side = 'sell'
            price = self.get_bid_ask()[1]
            print(self.contract_format(self.S * self.position_signal))
            self.order = asyncio.get_event_loop().run_until_complete(self.priv_api(json.dumps(self.order_msg(price, side, self.contract_format(self.S * self.position_signal)))))
            self.order_side = side
            self.order_price = price
        else:
            print('Have no orders, get signal to buy!\n')
            side = 'buy'
            price = self.get_bid_ask()[0]
            print(self.contract_format(self.S * np.abs(self.position_signal)))
            self.order = asyncio.get_event_loop().run_until_complete(self.priv_api(json.dumps(self.order_msg(price, side, self.contract_format(self.S * np.abs(self.position_signal))))))
            self.order_side = side
            self.order_price = price
        
    def trade(self):
        """Функция для тороговли."""
        while True:
            self.update()
            self.send_order()
            sleep(self.sleep_time)

In [ ]:
url = 'wss://www.deribit.com/ws/api/v2'
clientId = "yourclientid"
clientSecret = "yoursecret"

In [75]:
currency = ['BTC','ETH']
instrument = ['BTC-9FEB24','ETH-9FEB24']

In [76]:
MM_bot = MM_robot(instrument[1], currency[1], url, clientId, clientSecret, 0.5)

In [77]:
MM_bot.get_position_signal()

-0.000826597

In [ ]:
MM_bot.trade()

Have no orders, get signal to sell!

35
Получил сигнал в том же направлении на продажу

Analyzing order...

Получил сигнал в том же направлении на продажу

Получил приказ переставить заявку на best_ask!

Analyzing order...

Получил сигнал в том же направлении на продажу

Analyzing order...

Получил сигнал в том же направлении на продажу

Analyzing order...

Получил сигнал в том же направлении на продажу

Получил приказ переставить заявку на best_ask!

Analyzing order...

Получил сигнал в том же направлении на продажу

Получил приказ переставить заявку на best_ask!

Analyzing order...

Получил сигнал в том же направлении на продажу

Получил приказ переставить заявку на best_ask!

Analyzing order...

Получил сигнал в том же направлении на продажу

Получил приказ переставить заявку на best_ask!

Analyzing order...

Получил сигнал в том же направлении на продажу

Получил приказ переставить заявку на best_ask!

Analyzing order...

Получил сигнал в том же направлении на продажу

Analyzing or

In [68]:
pd.DataFrame(MM_bot.data_dict).to_excel("data_ETH.xlsx")

Task exception was never retrieved
future: <Task finished name='Task-2961' coro=<MM_robot.priv_api() done, defined at /tmp/ipykernel_18558/1154505912.py:36> exception=KeyboardInterrupt()>
Traceback (most recent call last):
  File "/home/daniil/.local/lib/python3.10/site-packages/IPython/core/interactiveshell.py", line 3553, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/tmp/ipykernel_18558/3202843701.py", line 1, in <module>
    MM_bot.trade()
  File "/tmp/ipykernel_18558/1154505912.py", line 388, in trade
    self.send_order()
  File "/tmp/ipykernel_18558/1154505912.py", line 365, in send_order
    self.analyze_order()
  File "/tmp/ipykernel_18558/1154505912.py", line 341, in analyze_order
    self.order = asyncio.get_event_loop().run_until_complete(self.priv_api(json.dumps(self.order_msg(best_bid, signal_side, self.contract_format(self.S * np.abs(self.position_signal))))))
  File "/home/daniil/.local/lib/python3.10/site-packages/nest_asyncio.py", line 93, 

ModuleNotFoundError: No module named 'openpyxl'